## Geo-Spatial Exercise in Python

## Exercise One: World Cities

### Preparation: 

* Download the First [Dataset with Latitude and Longitude](https://www.maxmind.com/en/free-world-cities-database) and revise the file name from worldcitiespop.txt to worldcitiespop.csv
* * Download the [data file of world countries] (https://github.com/python-visualization/folium/tree/master/examples/data) 

In [1]:
# Click Worldcitiespop.csv "Insert to code" -> "Insert SparkSession DataFrame" to generate the initial code

In [17]:
df_data_1.printSchema()
df_data_1.show(5)
df_data_1.createOrReplaceTempView("city")

root
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- AccentCity: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Population: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

+-------+----------+----------+------+----------+----------+---------+
|Country|      City|AccentCity|Region|Population|  Latitude|Longitude|
+-------+----------+----------+------+----------+----------+---------+
|     ad|     aixas|     Aix�s|    06|      null|42.4833333|1.4666667|
|     ad|aixirivali|Aixirivali|    06|      null|42.4666667|      1.5|
|     ad|aixirivall|Aixirivall|    06|      null|42.4666667|      1.5|
|     ad| aixirvall| Aixirvall|    06|      null|42.4666667|      1.5|
|     ad|  aixovall|  Aixovall|    06|      null|42.4666667|1.4833333|
+-------+----------+----------+------+----------+----------+---------+
only showing top 5 rows



In [88]:
cityDF = spark.sql("select Country, City, Population, Latitude, Longitude from city where Population is not null and Latitude is not null and Longitude is not null")

cityDF.printSchema()
cityDF.show(5)

root
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Population: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

+-------+----------------+----------+----------+---------+
|Country|            City|Population|  Latitude|Longitude|
+-------+----------------+----------+----------+---------+
|     ad|andorra la vella|     20430|      42.5|1.5166667|
|     ad|         canillo|      3292|42.5666667|      1.6|
|     ad|          encamp|     11224|42.5333333|1.5833333|
|     ad|      la massana|      7211|     42.55|1.5166667|
|     ad|    les escaldes|     15854|      42.5|1.5333333|
+-------+----------------+----------+----------+---------+
only showing top 5 rows



In [89]:
print "Total Cities: "+str(cityDF.count())

Total Cities: 14683


In [92]:
df=cityDF.withColumn("Latitude",cityDF.Latitude.cast("integer")).withColumn("Longitude",cityDF.Longitude.cast("integer")).withColumn("Population",cityDF.Population.cast("integer"))
df=df.filter(df.Population>2000000).orderBy("Population", ascending=False)
print "Total cities over 2M:" + str(df.count())
df.show(20)

Total cities over 2M:44
+-------+--------------+----------+--------+---------+
|Country|          City|Population|Latitude|Longitude|
+-------+--------------+----------+--------+---------+
|     cn|      shanghai|  14608512|      31|      121|
|     br|     sao paulo|  10021437|     -23|      -47|
|     cd|      kinshasa|   7787832|      -4|       15|
|     eg|         cairo|   7734602|      30|       31|
|     cn|        peking|   7480601|      40|      116|
|     gb|        london|   7421228|      52|        0|
|     co|        bogota|   7102602|       5|      -74|
|     bd|         dhaka|   6493177|      24|       90|
|     br|rio de janeiro|   6023742|     -23|      -43|
|     cl|      santiago|   4837248|     -33|      -71|
|     ca|       toronto|   4612187|      44|      -79|
|     au|        sydney|   4394585|     -34|      151|
|     cn|         wuhan|   4184206|      31|      114|
|     cn|     chongqing|   3967028|      30|      107|
|     cn|          xian|   3953191|      

In [93]:
descDF = df.describe("Population", "Latitude", "Longitude" )
descDF.show()
city_data = df.toJSON().collect()

+-------+------------------+------------------+-----------------+
|summary|        Population|          Latitude|        Longitude|
+-------+------------------+------------------+-----------------+
|  count|                44|                44|               44|
|   mean|4163182.9318181816|19.045454545454547|38.61363636363637|
| stddev| 2506839.151472863| 26.18268071752825|78.27299154289426|
|    min|           2000001|               -38|              -79|
|    max|          14608512|                53|              151|
+-------+------------------+------------------+-----------------+



In [26]:
!pip install folium

In [94]:
import folium

descJSON = descDF.toJSON().collect()
meanJSON = json.loads(descJSON[1])

lat = meanJSON["Latitude"];
lng = meanJSON["Longitude"];

tileset = r'http://{s}.tile.osm.org/{z}/{x}/{y}.png'
map_1 = folium.Map(location=[lat, lng], zoom_start=2, 
                 tiles=tileset, attr='Center ')

folium.Marker([lat, lng], popup='Cities with more than 2M people',
                   icon = folium.Icon(icon = 'cloud' ,color = 'green')).add_to(map_1)
import json

for row in city_data:
    city = json.loads(row)
    folium.Marker([city['Latitude'], city['Longitude']], popup=city['City']+":"+str(city['Population'])).add_to(map_1)
map_1


In [95]:
df_country = sc.textFile('swift://Citi.' + name + '/world-countries.json').collect()
df_country_str = ''.join(df_country)

In [96]:
map_2 = folium.Map(location=[lat, lng],
                   tiles='Mapbox Bright', zoom_start=2)
map_2.choropleth(geo_str=df_country_str,  legend_name='World Cities',highlight=True, reset=True)

for row in city_data:
    city = json.loads(row)
    folium.RegularPolygonMarker(location=[city['Latitude'], city['Longitude']], popup=city['City']+":"+str(city['Population']),
                   fill_color='#769d96', number_of_sides=6, radius=10).add_to(map_2)
map_2

## Scenario Two: Map by US State, County 

### Preparation:

* Download the [data files with US states county information](https://github.com/python-visualization/folium/tree/master/examples/data) 
* Dowbkiad the [US zipcode information](https://gist.github.com/jefffriesen/6892860)

In [97]:

df_unemployment = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://Citi.' + name + '/US_Unemployment_Oct2012.csv')
df_unemployment = df_unemployment.withColumn('Unemployment', df_unemployment.Unemployment.cast('float'))
df_unemployment.printSchema()
df_unemployment.show(5)


root
 |-- State: string (nullable = true)
 |-- Unemployment: float (nullable = true)

+-----+------------+
|State|Unemployment|
+-----+------------+
|   AL|         7.1|
|   AK|         6.8|
|   AZ|         8.1|
|   AR|         7.2|
|   CA|        10.1|
+-----+------------+
only showing top 5 rows



In [98]:
state_data = df_unemployment.toPandas()
df_state = sc.textFile('swift://Citi.' + name + '/us-states.json').collect()
df_state_str = ''.join(df_state)

In [99]:
import folium

map_3 = folium.Map(location=[48, -102], zoom_start=3)
map_3.choropleth(
    geo_str=df_state_str, 
    data=state_data,
    columns=['State', 'Unemployment'],
    threshold_scale=[5, 6, 7, 8, 9,10],
    key_on='feature.id',
    fill_color='YlGn', fill_opacity=0.7, line_opacity=0.5,
    legend_name='Unemployment Rate (%)',
    highlight=True,
    reset=True)
map_3

In [100]:
from pyspark.sql.functions import concat, col, lit

df_county_unemployment = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://Citi.' + name + '/us_county_data.csv')
df_county_unemployment = df_county_unemployment.withColumn("Unemployment_rate_2011", df_county_unemployment.Unemployment_rate_2011.cast('float'))  
df_county_unemployment.printSchema()
df_county_unemployment.show(5)


root
 |-- FIPS_Code: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Area_name: string (nullable = true)
 |-- Civilian_labor_force_2011: string (nullable = true)
 |-- Employed_2011: string (nullable = true)
 |-- Unemployed_2011: string (nullable = true)
 |-- Unemployment_rate_2011: float (nullable = true)
 |-- Median_Household_Income_2011: string (nullable = true)
 |-- Med_HH_Income_Percent_of_StateTotal_2011: string (nullable = true)

+---------+-----+--------------+-------------------------+-------------+---------------+----------------------+----------------------------+----------------------------------------+
|FIPS_Code|State|     Area_name|Civilian_labor_force_2011|Employed_2011|Unemployed_2011|Unemployment_rate_2011|Median_Household_Income_2011|Med_HH_Income_Percent_of_StateTotal_2011|
+---------+-----+--------------+-------------------------+-------------+---------------+----------------------+----------------------------+-------------------------------------

In [101]:
df = df_county_unemployment.toPandas()

def set_id(fips):
    '''Modify FIPS code to match GeoJSON property'''
    if fips == '0':
        return None
    elif len(fips) <= 4:
        return ''.join(['0500000US0', fips])
    else:
        return ''.join(['0500000US', fips])

#Apply set_id, drop NaN
df['GEO_ID'] = df['FIPS_Code'].apply(set_id)
df = df.dropna()


In [110]:
df_county = sc.textFile('swift://Citi.' + name + '/us_counties_20m_topo.json').collect()
df_county_str = ''.join(df_county)
data = json.loads(df_county_str)

In [111]:
map_4 = folium.Map(location=[48, -102], zoom_start=4)
folium.TopoJson(data, 'objects.us_counties_20m',name='topojson').add_to(map_4)
map_4

In [112]:

map_5 = folium.Map(location=[48, -102], zoom_start=5)
map_5.choropleth(geo_str=data, data_out='data1.json', data=df,
               columns=['GEO_ID', 'Unemployment_rate_2011'],
               key_on='feature.id',
               threshold_scale=[0, 5, 7, 9, 11, 13],
               fill_color='YlGnBu', line_opacity=0.3,
               legend_name='Unemployment Rate 2011 (%)',
               highlight=True,
               reset=True,
               topojson='objects.us_counties_20m')
map_5

In [113]:
df_zipcode = sc.textFile('swift://Citi.' + name + '/zips_us_topo.json').collect()
df_zipcode_str = ''.join(df_zipcode)
data = json.loads(df_zipcode_str)